In [14]:
%run 2gis_functions.ipynb

Ready: 2023-04-05 22:57:39


In [4]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()
spark

In [23]:
@show_time
def count_itogo_users(*files):
    file_path = "\\".join(files[0].split("\\")[:-1]) + "\\" + files[0].split("\\")[-2]
    d = {}
    for file_path in files:
        df = dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['user', 'rubricName', 'date']]
        year = file_path.split('_')[-2]
        user_cnt_year = df.user.nunique().compute()
        d[year] = user_cnt_year
        print(year, user_cnt_year)
    df_list = [dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['rubricName', 'user']] for file_path in files]
    df_all = dd.concat(df_list)
    cnt_user_total = df_all.user.nunique().compute()
    d['Итого'] = cnt_user_total
    print(cnt_user_total)
    cnt_df = pd.DataFrame([d], index=["Итого"])
    cnt_df.to_csv(f'{file_path}_cnt_user_itogo.csv', index=False)
    return cnt_df
        
def count_users_requests(file_path):
    try:
        cnt_user_grouped_year = dd.read_csv(f'{file_path}_count_users_requests.csv')
        print('read ' + f'{file_path}_count_users_requests.csv')
    except:
        year = file_path.split('_')[-2]
        requests_name = f'Запросы {year}'
        users_name = f'Пользователи {year}'
        df = dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['user', 'rubricName', 'date']]
        cnt_user_grouped = df.groupby('rubricName').user.nunique().to_frame()
        cnt_req_grouped = df.groupby('rubricName').date.count().to_frame()
        cnt_user_grouped_year = (cnt_req_grouped
                      .merge(cnt_user_grouped, left_on='rubricName', right_on='rubricName')
                      .rename(columns={'date':requests_name, 'user':users_name})
                      .sort_values(requests_name, ascending=False)
                      .fillna(0)
                      .astype(int)
                      )
        cnt_user_grouped_year.to_csv(f'{file_path}_count_users_requests.csv', single_file=True)
    finally:
        df = dd.read_csv(f'{file_path}_count_users_requests.csv')

    return df

@show_time
def count_total_users(*files):
    file_path = "\\".join(files[0][0].split("\\")[:-1]) + "\\" + files[0][0].split("\\")[-2]
    try:
        cnt_user_grouped_total = dd.read_csv("{}_count_total_users.csv".format(file_path))
        print('read ' + "{}_count_total_users.csv".format(file_path))
    except:
        df_list = [dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['rubricName', 'user']] for file_path in files]
        df_all = dd.concat(df_list)
        cnt_user_grouped_total = df_all.groupby('rubricName').user.nunique().to_frame().rename(columns={'user':'Пользователи, итого'})
        cnt_user_grouped_total.to_csv(f'{file_path}_count_total_users.csv', single_file=True)
    finally:
        df = dd.read_csv("{}_count_total_users.csv".format(file_path))
    return df

@show_time
def make_base_counted_file(*files):
    file_path = "\\".join(files[0].split("\\")[:3]) + "\\" + files[0].split("\\")[2]
    df = count_users_requests(files[0])
    for file in files[1:]:
        df = df.merge(count_users_requests(file), left_on='rubricName', right_on='rubricName', how='outer')
    df = df.merge(count_total_users(files), left_on='rubricName', right_on='rubricName', how='outer')
    df.to_csv(f'{file_path}_master_file.csv', single_file=True, index=False)
    return df

from dask.diagnostics import ProgressBar
ProgressBar().register()

In [25]:
d = count_itogo_users(
"D:\файлы_2гис_2023\Баку\Баку_2020_975740.csv",
"D:\файлы_2гис_2023\Баку\Баку_2021_975746.csv",
"D:\файлы_2гис_2023\Баку\Баку_2022_975747.csv",
"D:\файлы_2гис_2023\Баку\Баку_2023_975748.csv",
)

[########################################] | 100% Completed | 6.39 ss
[########################################] | 100% Completed | 6.50 s
[########################################] | 100% Completed | 6.61 s
2020 19220
[########################################] | 100% Completed | 685.51 ms
[########################################] | 100% Completed | 788.10 ms
[########################################] | 100% Completed | 895.66 ms
2021 5337
[########################################] | 100% Completed | 1.98 ss
[########################################] | 100% Completed | 2.08 s
[########################################] | 100% Completed | 2.16 s
2022 10975
[########################################] | 100% Completed | 2.34 ss
[########################################] | 100% Completed | 2.38 s
[########################################] | 100% Completed | 2.43 s
2023 20240
[########################################] | 100% Completed | 3.98 ss
[########################################] | 10

In [26]:
d

,2020,2021,2022,2023,Итого
Итого,19220,5337,10975,20240,51622


In [8]:
d = count_itogo_users(
"D:\файлы_2гис_2023\Астана\Астана_2020_975007.csv",
"D:\файлы_2гис_2023\Астана\Астана_2021_975009.csv",
"D:\файлы_2гис_2023\Астана\Астана_2022_975011.csv",
"D:\файлы_2гис_2023\Астана\Астана_2023_975039.csv",

)

[########################################] | 100% Completed | 309.06 s
[########################################] | 100% Completed | 309.16 s
[########################################] | 100% Completed | 309.27 s
[########################################] | 100% Completed | 309.37 s
[########################################] | 100% Completed | 309.48 s
2020 790636
[########################################] | 100% Completed | 343.20 s
[########################################] | 100% Completed | 343.31 s
[########################################] | 100% Completed | 343.42 s
[########################################] | 100% Completed | 343.52 s
[########################################] | 100% Completed | 343.63 s
2021 828082
[########################################] | 100% Completed | 397.01 s
[########################################] | 100% Completed | 397.12 s
[########################################] | 100% Completed | 397.22 s
[########################################] | 100% Com

# {'2020': 790636,
 '2021': 828082,
 '2022': 1032340,
 '2023': 1313830,
 'total': 2723378}

In [ ]:
# {'2020': 790636,
 '2021': 828082,
 '2022': 1032340,
 '2023': 1313830,
 'total': 2723378}

In [8]:
d = {'2020': 790636,
 '2021': 828082,
 '2022': 1032340,
 '2023': 1313830,
 'Итого': 2723378}

In [10]:
import pandas as pd

,2020,2021,2022,2023,Итого
Итого,790636,828082,1032340,1313830,2723378
